### Decision Tree

In [1]:
import pandas as pd
import numpy as np
from numpy import log2 as log

In [2]:
Outlook = 'overcast,overcast,overcast,overcast,rainy,rainy,rainy,rainy,rainy,sunny,sunny,sunny,sunny,sunny'.split(',')
temp = 'hot,cool,mild,hot,mild,cool,cool,mild,mild,hot,hot,mild,cool,mild'.split(',')
humidity = 'high,normal,high,normal,high,normal,normal,normal,high,high,high,high,normal,normal'.split(',')
windy = 'FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE'.split(',')
play = 'yes,yes,yes,yes,yes,yes,no,yes,no,no,no,no,yes,yes'.split(',')

In [3]:
dataset = {'Outlook': Outlook,
           'temp': temp,
           'humidity': humidity,
           'windy': windy,
           'play': play}
df = pd.DataFrame(dataset, columns=['Outlook', 'temp', 'humidity', 'windy', 'play'])

In [4]:
df

,Outlook,temp,humidity,windy,play
0,overcast,hot,high,FALSE,yes
1,overcast,cool,normal,TRUE,yes
2,overcast,mild,high,TRUE,yes
3,overcast,hot,normal,FALSE,yes
4,rainy,mild,high,FALSE,yes
5,rainy,cool,normal,FALSE,yes
6,rainy,cool,normal,TRUE,no
7,rainy,mild,normal,FALSE,yes
8,rainy,mild,high,TRUE,no
9,sunny,hot,high,FALSE,no


In [6]:
entropy_node = 0
for value in values:
    fraction = df.play.value_counts()[value]/len(df.play)
    entropy_node += -fraction*np.log2(fraction)
print(f'entropy_node: {entropy_node}')
print(entropy_node)

NameError: name 'value' is not defined

In [7]:
#windy
target_variables = df.play.unique()
variables = df['windy'].unique()
print(variables, target_variables)

['FALSE' 'TRUE'] ['yes' 'no']


In [8]:
entropy_windy = 0
for variable in variables:
    entropy_each_feature = 0
    for target_variable in target_variables:
        num = len(df['windy'][df['windy']==variable][df.play == target_variable])
        den = len(df['windy'][df['windy']==variable])
        fraction = num/den
        entropy_each_feature += -fraction*np.log2(fraction)
    fraction2 = den/len(df)
    entropy_windy += fraction2*entropy_each_feature
print(entropy_windy)

0.8921589282623617


In [9]:
eps = 0.000000000000001

In [10]:
def ent (df, attribute):
    target_variables = df.play.unique()
    variables = df[attribute].unique()
    
    
    entropy_attribute = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df.play == target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy_each_feature += -fraction*np.log2(fraction+eps)
        fraction2 = den/len(df)
        entropy_attribute += fraction2*entropy_each_feature
        
    return (entropy_attribute) 

In [11]:
df.keys()[:-1]

Index(['Outlook', 'temp', 'humidity', 'windy'], dtype='object')

In [12]:
list_entropy = {k: ent(df, k) for k in df.keys()[:-1]}
print(list_entropy)

{'Outlook': 0.6935361388961896, 'temp': 0.9110633930116735, 'humidity': 0.7884504573082867, 'windy': 0.8921589282623588}


In [13]:
# Information Gain
def ig (a, b):
    return (a-b)

In [14]:
list_ig = {k: ig(entropy_node, list_entropy[k]) for k in list_entropy}
print(list_ig)

{'Outlook': -0.6935361388961896, 'temp': -0.9110633930116735, 'humidity': -0.7884504573082867, 'windy': -0.8921589282623588}


# Decision Tree with skearn

In [15]:
import pandas as pd
col_names = ['pregnants', 'glucose', 'bp', 'skin', 'insulin', 'bmi', 'pedigree', 'age', 'label']
dframe = pd.read_csv('../dataset/Pima.csv', header = None, names = col_names)
dframe.head()

,pregnants,glucose,bp,skin,insulin,bmi,pedigree,age,label
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [16]:
#Split Train and test samples
X = dframe.drop(['label'], axis = 1)
y = dframe.label
print(X.shape, y.shape)

(768, 8) (768,)


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [18]:
#Let's build the decision tree model
from sklearn.tree import DecisionTreeClassifier
Model = DecisionTreeClassifier()
Model_tree = Model.fit(X_train, y_train)

In [19]:
#Prediction on test sample
y_pred = Model_tree.predict(X_test)
y_pred_proba = Model_tree.predict_proba(X_test)
print(y_pred)

[0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 0 1 0 1 1 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 0 1 0 0 1 1 1 1 1 0 0
 1 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 1 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 1 0 0 0
 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0]


In [20]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, Model.predict(X_test))

array([[113,  33],
       [ 43,  42]], dtype=int64)

# Let's Apply our GridSearchCV for Decision Tree

In [21]:
from sklearn.model_selection import GridSearchCV
params_grid = {'criterion': ['gini', 'entropy'], 'max_depth': np.arange(5, 20)}

In [22]:
Model = DecisionTreeClassifier()
Model_Grid = GridSearchCV(Model, params_grid, cv=5)
Model_Grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])})

In [23]:
Model_Grid.best_score_

0.713343717549325

In [24]:
Model_Grid.best_params_

{'criterion': 'entropy', 'max_depth': 7}